In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('stock_dataset.db')
cursor = conn.cursor()

df = pd.read_csv('data/hackathon_sample_v2.csv')

df.to_sql('hackathon_sample_v2', conn, if_exists='replace', index=False)

nasdaq_data = pd.read_csv('nasdaq-listed.csv')
nasdaq_data.to_sql('nasdaq_listed', conn, if_exists='replace', index=False)

nyse_data = pd.read_csv('nyse-listed.csv')
nyse_data.to_sql('nyse_listed', conn, if_exists='replace', index=False)

other_data = pd.read_csv('other-listed.csv')
other_data.to_sql('other_listed', conn, if_exists='replace', index=False)
cursor.execute('ALTER TABLE nasdaq_listed RENAME COLUMN Symbol TO "ACT Symbol";')

removed_outdated_tickers = pd.read_sql_query('SELECT * FROM hackathon_sample_v2 WHERE stock_ticker IN (SELECT "ACT Symbol" FROM nasdaq_listed UNION SELECT "ACT Symbol" FROM nyse_listed UNION SELECT "ACT Symbol" FROM other_listed)', conn)
removed_outdated_tickers.to_sql('tickerremoved_hackathon_sample_v2', conn, if_exists='replace', index=False)

# all companies having complete 12 months data from 2019 to 2023 atleast
atleast_five_year_companies = pd.read_sql_query("SELECT DISTINCT stock_ticker FROM tickerremoved_hackathon_sample_v2 WHERE year IN (2019, 2020, 2021, 2022, 2023) GROUP BY stock_ticker HAVING COUNT(*) = 60;",conn)

atleast_five_year_companies.to_sql('atleast_five_year_companies', conn, if_exists='replace', index=False)
filtered_df = pd.read_sql_query("Select year, month, intrinsic_value,stock_exret, stock_ticker,comp_name,be_me,ni_me,fcf_me,betadown_252d, ni_ar1, z_score, ebit_sale, at_turnover, market_equity from tickerremoved_hackathon_sample_v2 where stock_ticker IN (SELECT stock_ticker FROM atleast_five_year_companies) order by stock_ticker, year, month ", conn)

filtered_df['prev_intrinsic_value'] = filtered_df['intrinsic_value'].shift(1)
filtered_df['next_intrinsic_value'] = filtered_df['intrinsic_value'].shift(-1)


filtered_df['prev_stock_exret'] = filtered_df['stock_exret'].shift(1)
filtered_df['next_stock_exret'] = filtered_df['stock_exret'].shift(-1)


filtered_df['prev_be_me'] = filtered_df['be_me'].shift(1)
filtered_df['next_be_me'] = filtered_df['be_me'].shift(-1)


filtered_df['prev_ni_me'] = filtered_df['ni_me'].shift(1)
filtered_df['next_ni_me'] = filtered_df['ni_me'].shift(-1)


filtered_df['prev_fcf_me'] = filtered_df['fcf_me'].shift(1)
filtered_df['next_fcf_me'] = filtered_df['fcf_me'].shift(-1)

filtered_df['prev_betadown_252d'] = filtered_df['betadown_252d'].shift(1)
filtered_df['next_betadown_252d'] = filtered_df['betadown_252d'].shift(-1)

filtered_df['prev_ni_ar1'] = filtered_df['ni_ar1'].shift(1)
filtered_df['next_ni_ar1'] = filtered_df['ni_ar1'].shift(-1)

filtered_df['prev_z_score'] = filtered_df['z_score'].shift(1)
filtered_df['next_z_score'] = filtered_df['z_score'].shift(-1)

filtered_df['prev_ebit_sale'] = filtered_df['ebit_sale'].shift(1)
filtered_df['next_ebit_sale'] = filtered_df['ebit_sale'].shift(-1)

filtered_df['prev_at_turnover'] = filtered_df['at_turnover'].shift(1)
filtered_df['next_at_turnover'] = filtered_df['at_turnover'].shift(-1)

filtered_df['prev_market_equity'] = filtered_df['market_equity'].shift(1)
filtered_df['next_market_equity'] = filtered_df['market_equity'].shift(-1)

filtered_df['market_equity'] = filtered_df.apply(
    lambda row: (row['prev_market_equity'] + row['next_market_equity']) / 2 if pd.isnull(row['market_equity']) and pd.notnull(row['prev_market_equity']) and pd.notnull(row['next_market_equity']) 
                else row['market_equity'],
    axis=1
)

filtered_df['intrinsic_value'] = filtered_df.apply(
    lambda row: (row['prev_intrinsic_value'] + row['next_intrinsic_value']) / 2 if pd.isnull(row['intrinsic_value']) and pd.notnull(row['prev_intrinsic_value']) and pd.notnull(row['next_intrinsic_value']) 
                else row['intrinsic_value'],
    axis=1
)

filtered_df['stock_exret'] = filtered_df.apply(
    lambda row: (row['prev_stock_exret'] + row['next_stock_exret']) / 2 if pd.isnull(row['stock_exret']) and pd.notnull(row['prev_stock_exret']) and pd.notnull(row['next_stock_exret'])
                else row['stock_exret'],
    axis=1
)

filtered_df['be_me'] = filtered_df.apply(
    lambda row: (row['prev_be_me'] + row['next_be_me']) / 2 if pd.isnull(row['be_me']) and pd.notnull(row['prev_be_me']) and pd.notnull(row['next_be_me'])
                else row['be_me'],
    axis=1
)

filtered_df['ni_me'] = filtered_df.apply(
    lambda row: (row['prev_ni_me'] + row['next_ni_me']) / 2 if pd.isnull(row['ni_me']) and pd.notnull(row['prev_ni_me']) and pd.notnull(row['next_ni_me'])
                else row['ni_me'],
    axis=1
)

filtered_df['fcf_me'] = filtered_df.apply(
    lambda row: (row['prev_fcf_me'] + row['next_fcf_me']) / 2 if pd.isnull(row['fcf_me']) and pd.notnull(row['prev_fcf_me']) and pd.notnull(row['next_fcf_me'])
                else row['fcf_me'],
    axis=1
)

filtered_df['betadown_252d'] = filtered_df.apply(
    lambda row: (row['prev_betadown_252d'] + row['next_betadown_252d']) / 2 if pd.isnull(row['betadown_252d']) and pd.notnull(row['prev_betadown_252d']) and pd.notnull(row['next_betadown_252d'])
                else row['betadown_252d'],
    axis=1
)

filtered_df['ni_ar1'] = filtered_df.apply(
    lambda row: (row['prev_ni_ar1'] + row['next_ni_ar1']) / 2 if pd.isnull(row['ni_ar1']) and pd.notnull(row['prev_ni_ar1']) and pd.notnull(row['next_ni_ar1'])
                else row['ni_ar1'],
    axis=1
)

filtered_df['z_score'] = filtered_df.apply(
    lambda row: (row['prev_z_score'] + row['next_z_score']) / 2 if pd.isnull(row['z_score']) and pd.notnull(row['prev_z_score']) and pd.notnull(row['next_z_score'])
                else row['z_score'],
    axis=1
)

filtered_df['ebit_sale'] = filtered_df.apply(
    lambda row: (row['prev_ebit_sale'] + row['next_ebit_sale']) / 2 if pd.isnull(row['ebit_sale']) and pd.notnull(row['prev_ebit_sale']) and pd.notnull(row['next_ebit_sale'])
                else row['ebit_sale'],
    axis=1
)

filtered_df['at_turnover'] = filtered_df.apply(
    lambda row: (row['prev_at_turnover'] + row['next_at_turnover']) / 2 if pd.isnull(row['at_turnover']) and pd.notnull(row['prev_at_turnover']) and pd.notnull(row['next_at_turnover'])
                else row['at_turnover'],
    axis=1
)

filtered_df['market_equity'] = filtered_df.apply(
    lambda row: (row['prev_market_equity'] + row['next_market_equity']) / 2 if pd.isnull(row['market_equity']) and pd.notnull(row['prev_market_equity']) and pd.notnull(row['next_market_equity'])
                else row['market_equity'],
    axis=1
)

# if feature is still null drop that row
filtered_df = filtered_df.dropna(subset=['intrinsic_value', 'market_equity', 'at_turnover', 'ebit_sale', 'z_score', 'ni_ar1', 'betadown_252d', 'fcf_me', 'ni_me', 'be_me', 'stock_exret'])

filtered_df['roic'] = filtered_df['ebit_sale'] * filtered_df['at_turnover']

filtered_df['bvps'] = filtered_df['be_me']*filtered_df['market_equity']


filtered_df['prev_bvps'] = filtered_df.groupby('stock_ticker')['bvps'].shift(1)
filtered_df['next_bvps'] = filtered_df.groupby('stock_ticker')['bvps'].shift(-1)


filtered_df['bvps_change'] = (filtered_df['bvps'] - filtered_df['prev_bvps']) / filtered_df['prev_bvps']

# Calculate the at_turnover change for sales growth change from the previous month
filtered_df['prev_at_turnover'] = filtered_df.groupby('stock_ticker')['at_turnover'].shift(1)
filtered_df['at_turnover_change'] = (filtered_df['at_turnover'] - filtered_df['prev_at_turnover']) / filtered_df['prev_at_turnover']

# Calculate the earnings to price  growth change from the previous month
filtered_df['prev_ni_me'] = filtered_df.groupby('stock_ticker')['ni_me'].shift(1)
filtered_df['ni_me_change'] = (filtered_df['ni_me'] - filtered_df['prev_ni_me']) / filtered_df['prev_ni_me']

# Calculate the FCF to price  growth change from the previous month
filtered_df['prev_fcf_me'] = filtered_df.groupby('stock_ticker')['fcf_me'].shift(1)
filtered_df['fcf_me_change'] = (filtered_df['fcf_me'] - filtered_df['prev_fcf_me']) / filtered_df['prev_fcf_me']

filtered_df.to_sql('filtered_hackathon_sample_v2', conn, if_exists='replace', index=False)
filtered_df.to_csv('filtered_stock_data_1.csv', index=False)

conn.commit()
conn.close()

In [4]:
import pandas as pd
import os

# Load the full dataset
df = pd.read_csv('filtered_stock_data_1.csv')

# Create a folder for company files
os.makedirs("data", exist_ok=True)

# Save each company separately as CSV
for company in df['stock_ticker'].unique():
    safe_company_name = company.replace("/", "_")  # Replace '/' with '_'
    company_df = df[df['stock_ticker'] == company]
    
    # Save as CSV instead of Parquet
    company_df.to_csv(f'data/{safe_company_name}.csv', index=False)

print("All company data saved separately as CSV.")

All company data saved separately as CSV.
